In [1]:
from mhcflurry import Class1AffinityPredictor
import pandas as pd
import numpy as np
import time
import glob
import seaborn as sns
import warnings

import matplotlib.pyplot as plt

from Bio import Entrez
Entrez.email="yq7@illinois.edu"

predictor = Class1AffinityPredictor.load()

In [2]:

ERR = pd.DataFrame([["N/A","N/A",-1.0,-1.0,-1.0,-1.0]],columns=['peptide', 'allele', 'prediction', 'prediction_low', 
                               'prediction_high', 'prediction_percentile'])
ERR

,peptide,allele,prediction,prediction_low,prediction_high,prediction_percentile
0,N/A,N/A,-1.0,-1.0,-1.0,-1.0


In [3]:
predictor.supported_alleles[:10]

['BoLA-6*13:01',
 'Eqca-1*01:01',
 'H-2-Db',
 'H-2-Dd',
 'H-2-Kb',
 'H-2-Kd',
 'H-2-Kk',
 'H-2-Ld',
 'HLA-A*01:01',
 'HLA-A*02:01']

### 1.Data

In [4]:
#read purity info
path_purity="ncomms9971-s2.xlsx"
purity = pd.read_excel(path_purity)
#purity
def purity_info(Sample_Id):
    result=purity["CPE"][purity["Sample ID"].str.contains(Sample_Id)]
    if(len(result)==0): return np.nan
    return result.iloc[0]
#purity_info("TCGA-L7-A56G-01")
#purity["Sample ID"].str.contains("TCGA-OR-A5J1-01")

In [5]:
#read copy number info
def log_minus_GL(row,minus_GL_tf):
    pos = int(row["Start_position"])
    ch = int(row["Chromosome"])
    #print(pos,ch)
    result=minus_GL_tf["Segment_Mean"][ (minus_GL_tf["End"]>=pos) &
                                (minus_GL_tf["Start"]<=pos) &
                                (minus_GL_tf["Chromosome"]==ch) ]
    if len(result)==0: return 1000
    if len(result)>=2: raise Exception("%d position matched!"%len(result))
    #print( result )
    
    return result.iloc[0]

In [6]:
test_path="ACC/ACC.rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data.data.txt"
def read_rna(path):
    #print (path)
    tf=pd.read_csv(path,sep="\t")
    return tf

def obtain_rna_data(data,sample_id,hugo_symbol="%invalid:N/A",gene_id="%invalid:N/A"):
    cols=[data.columns[0]]
    if (len(hugo_symbol)<=2): hugo_symbol="%invalid:N/A"
    if (gene_id<=0): gene_id="%invalid:N/A"
    for col in data.columns:
        if sample_id in col:
            cols.append(col)
    if (len(cols)!=2): return -1.0
    tf=data[cols] 
    tf=tf[ tf[data.columns[0]].str.contains(hugo_symbol) | tf[data.columns[0]].str.contains(str(gene_id))]
    #return tf
    if (len(tf) !=1): return -1.0
    return tf.iloc[0,1]
tf=read_rna(test_path)
obtain_rna_data(tf,"TCGA-OR-A5JM-01","HRNR",0)

1.7241

In [7]:
HLA_path = "OptiTypeCallsHLA_20171207.tsv"
HLA_Table = pd.read_csv(HLA_path)
SixHLAs=HLA_Table.columns[:6]
def obtain_HLAs(patient_id,HLA_N):
    tf=HLA_Table[HLA_Table["aliquot_id"].str.contains(patient_id)]
    if(len(tf)<1): raise Exception("Not Found")
    return tf[HLA_N].iloc[0]


In [8]:
def read_rename_file(Type):
    Dict={}
    with open("Types/%s"%Type) as f:
        for line in f.readlines():
            if line[0]=="#": 
                continue
            line=line.rstrip("\n").split(":")
            Dict[line[1]]=line[0]
    return Dict

path="BLCA/gdac.broadinstitute.org_BLCA.Mutation_Packager_Calls.Level_3.2016012800.0.0/TCGA-BL-A0C8-01.maf.txt"
tf=pd.read_csv(path,sep="\t")
tf["cDNA_Change"]

0                  c.2352C>T
1                  c.2012G>C
2                   c.727C>T
3                   c.110C>T
4                  c.1605C>G
5                    c.36C>T
6                  c.1014G>C
7                     c.4G>C
8                   c.617C>T
9                    c.94G>C
10                 c.1988A>T
11                 c.3297C>T
12                  c.632C>G
13                  c.811C>G
14                  c.130G>A
15                 c.3622G>C
16                  c.613G>A
17                 c.1068C>T
18                   c.18C>A
19                 c.4367A>G
20                 c.2798C>A
21                  c.262G>A
22                  c.977G>C
23                  c.424G>A
24                  c.454G>A
25                  c.316G>A
26                c.10688A>G
27              c.414_splice
28                    c.7C>T
29                 c.1807G>A
               ...          
144                 c.286G>A
145                 c.284C>T
146                 c.938C>T
147           

In [9]:

def read_and_filter(minus_GL_path,sample_path,Type):
    
    cn = pd.read_csv(minus_GL_path,sep="\t")
    sample_id = sample_path.split("/")[-1].rstrip(".maf.txt")
    print("reading %s"%sample_id)
    minus_GL_tf=cn[cn["Sample"].str.contains(sample_id)]

    tf=pd.read_csv(sample_path,sep="\t")

    Rename_dict=read_rename_file(Type)
    tf=tf.rename(columns=Rename_dict)
    tf=tf.rename(columns={"Start_Position":"Start_position"})
    
    tf=tf[(tf["Variant_Type"]=="SNP") & (tf["Variant_Classification"]=="Missense_Mutation")] 
    if 'X' in (set(tf["Chromosome"].tolist())) or 'Y' in (set(tf["Chromosome"].tolist())):
        tf= tf[(tf["Chromosome"]!="X") & (tf["Chromosome"]!="Y")]
    
    
    pur=purity_info(sample_id)
    if ((pur is None) or np.isnan(pur)): raise Exception("NaN purity")

    if("VAF" not in tf.columns): 
        if("Tot" not in tf.columns): tf["Tot"]=tf["Var"]+tf["Ref"]
        tf["VAF"]=tf["Var"]/tf["Tot"]
    
    if(max(tf["VAF"].tolist())>=5):tf["VAF"]/=100.0
    
    tf["CCF"]=(tf["VAF"]*2.0)/pur

    tf["cn-gl"]=tf.apply( (lambda x:log_minus_GL(x,minus_GL_tf)) ,axis=1)
    tf=tf[(tf["cn-gl"]>=-0.3) & (tf["cn-gl"]<=0.3)]
    if (tf.shape[0]==0): raise Exception("No CN information")
    tf["Type"]=Type
    tf["Sample_Id"]=sample_id
    tf["Chromosome"] = tf.apply( (lambda x: int(x["Chromosome"])), axis=1 )
    tf["Loc"]=tf["Start_position"]
    
    for HLA_N in SixHLAs:
        tf[HLA_N]=obtain_HLAs(sample_id,HLA_N)

    tf=tf[(tf["Tid"]!=".") & (tf["Tid"].apply(lambda x:str(x)!="nan")) ]
    return tf[ ["Type","Sample_Id","Hugo_Symbol","Entrez_Gene_Id","Chromosome","Loc","Tid","NC","AAC","cn-gl","VAF","CCF"]+list(SixHLAs) ]

minus_GL_path="ACC/ACC.snp__genome_wide_snp_6__broad_mit_edu__Level_3__segmented_scna_minus_germline_cnv_hg19__seg.seg.txt"
path="ACC/gdac.broadinstitute.org_ACC.Mutation_Packager_Calls.Level_3.2016012800.0.0/TCGA-OR-A5JM-01.maf.txt"
read_and_filter(minus_GL_path,path,"ACC")#[:5]

reading TCGA-OR-A5JM-01


,Type,Sample_Id,Hugo_Symbol,Entrez_Gene_Id,Chromosome,Loc,Tid,NC,AAC,cn-gl,VAF,CCF,A1,A2,B1,B2,C1,C2
4,ACC,TCGA-OR-A5JM-01,HRNR,388697,1,152188235,NM_001009931,c.G5870A,p.G1957D,0.0346,0.023891,0.048293,A*02:01,A*02:01,B*35:01,B*07:02,C*07:02,C*04:01
6,ACC,TCGA-OR-A5JM-01,PEAR1,375033,1,156878718,NM_001080471,c.G1301A,p.C434Y,0.0346,0.357143,0.721938,A*02:01,A*02:01,B*35:01,B*07:02,C*07:02,C*04:01
7,ACC,TCGA-OR-A5JM-01,TOR3A,64222,1,179051300,NM_022371,c.T37C,p.F13L,0.0346,0.571429,1.155101,A*02:01,A*02:01,B*35:01,B*07:02,C*07:02,C*04:01
9,ACC,TCGA-OR-A5JM-01,RASSF5,83593,1,206730911,NM_182665,c.G10T,p.D4Y,0.0294,0.348214,0.703890,A*02:01,A*02:01,B*35:01,B*07:02,C*07:02,C*04:01
14,ACC,TCGA-OR-A5JM-01,ANKRD16,54522,10,5931230,NM_001009943,c.G88A,p.G30R,0.0286,0.875000,1.768749,A*02:01,A*02:01,B*35:01,B*07:02,C*07:02,C*04:01
23,ACC,TCGA-OR-A5JM-01,SYT8,90019,11,1858572,NM_138567,c.C1117T,p.R373W,0.0344,0.642857,1.299489,A*02:01,A*02:01,B*35:01,B*07:02,C*07:02,C*04:01
24,ACC,TCGA-OR-A5JM-01,OR51F1,256892,11,4790948,NM_001004752,c.G200T,p.R67M,0.0344,0.040816,0.082507,A*02:01,A*02:01,B*35:01,B*07:02,C*07:02,C*04:01
25,ACC,TCGA-OR-A5JM-01,OR51F1,256892,11,4790951,NM_001004752,c.T197C,p.F66S,0.0344,0.041096,0.083072,A*02:01,A*02:01,B*35:01,B*07:02,C*07:02,C*04:01
26,ACC,TCGA-OR-A5JM-01,AGBL2,79841,11,47712028,NM_024783,c.G1231C,p.A411P,0.0344,0.127389,0.257507,A*02:01,A*02:01,B*35:01,B*07:02,C*07:02,C*04:01
32,ACC,TCGA-OR-A5JM-01,FAM86C1,55199,11,71498671,NM_152563,c.G89C,p.R30P,0.0344,0.285714,0.577551,A*02:01,A*02:01,B*35:01,B*07:02,C*07:02,C*04:01


### 2 Peptide

In [10]:

def fetch_protein_TID(Id):
    if (len(glob.glob("Protein/"+Id))>0):
        with open("Protein/"+Id) as f:
            return f.read().rstrip("\n")
    
    
    time.sleep(0.34)
    handle = Entrez.efetch(db = 'nucleotide', id = Id, retmode = 'fasta', rettype = 'text')
    data = handle.read()
    handle.close()
    k=data.find("mol aa")
    while(data[k]!="\""): k+=1
    k2=k+1
    while(data[k2]!="\""): k2+=1
    seq=data[k+1:k2].replace("\n","")
    
    with open("Protein/"+Id,"w") as f:
        f.write(seq)

    return seq

print(len(fetch_protein_TID("NM_002457")))



5289


In [11]:
def fetch_protein_Ensembl(Id):
    if (len(glob.glob("Protein/"+Id))>0):
        with open("Protein/"+Id) as f:
            return f.read().rstrip("\n")
    
    time.sleep(0.34)
    handle=Entrez.esearch(db="protein", term=Id)
    out=Entrez.read(handle)
    handle.close()
    Pid=out["IdList"][0]
    time.sleep(0.34)
    handle = Entrez.efetch(db = 'protein', id = Pid, retmode = 'fasta', rettype = 'text')
    data = handle.read()
    handle.close()
    k=data.find("mol aa")
    while(data[k]!="\""): k+=1
    k2=k+1
    while(data[k2]!="\""): k2+=1
    seq=data[k+1:k2].replace("\n","")
    
    with open("Protein/"+Id,"w") as f:
        f.write(seq)

    return seq

out=fetch_protein_Ensembl("ENST00000269081")


In [12]:
def obtain_peptides(Transcript_Id,AAChange,l=8,r=11):
    #print("obtaining peptides for transcript ID %s with AA change %s"%(Transcript_Id,AAChange))
    
    if(Transcript_Id[0]=="E"):seq=fetch_protein_Ensembl(Transcript_Id)
    else: seq=fetch_protein_TID(Transcript_Id)
    AAChange=AAChange.lstrip("p.")
    AAFrom=AAChange[0]
    AATo=AAChange[-1]
    AAPos=int(AAChange[1:-1])
    #print((seq[0]))
    if (AAPos>len(seq)): raise Exception("AA change position out of range")
    if (seq[AAPos-1]!=AAFrom): raise Exception("AA not match")
    seq=seq[:AAPos-1]+AATo+seq[AAPos:]
    peptides=[]
    for i in range(l,r+1):
        for j in range(i):
            if AAPos-j-1 < 0 or AAPos-j+i-1 > len(seq) : continue
            peptides.append(seq[AAPos-j-1:AAPos-j+i-1])
    return peptides


#obtain_peptides("ENST00000269081","p.M10A","ens")
#obtain_peptides("NM_020888","p.G285A")




### 3.Test

In [13]:
predictor = Class1AffinityPredictor.load()
def predict_binding(Transcript_Id,AAChange,allele):#cri="min",sel="all",allele="HLA-A*02:01"):
    #return 1.0
    #print("HLA=%s"%(allele))
    #if allele not in predictor.supported_alleles:
    #    return -1.0
    #File_Name="Binding_Rank/%s-%s(%s).csv"%(Transcript_Id,AAChange,allele)
    for i in range(10):
        #print(Transcript_Id)
#         if (len(glob.glob(File_Name))>0):
#             tf=pd.read_csv(File_Name)
#             print(tf)
        if (Transcript_Id[0]=="E" and i==1): break;
        TID = Transcript_Id if i == 0 else Transcript_Id + ".%d"%i
        try:
            peptides=obtain_peptides(TID,AAChange)
            #print(peptides)
        except Exception as inst:
            #print(inst)
            if ("Bad Request" in str(inst)): break
            continue
        #print(peptides)
        try:
            tf=predictor.predict_to_dataframe(allele=allele,peptides=peptides)
        except Exception as inst:
            break
        if(len(tf)<38): break
        lmtf=tf.iloc[[3,8+4,8+9+4,8+9+10+5]]
        rmtf=tf.iloc[[4,8+4,8+9+5,8+9+10+5]]
        tf=pd.concat([lmtf,rmtf])
        return min(tf["prediction_percentile"].tolist())
        #sum(tf["prediction_percentile"].tolist())/len(tf["prediction_percentile"].tolist())
        '''
        if(cri=="min"): return min(tf["prediction_percentile"].tolist())
        else: return sum(tf["prediction_percentile"].tolist())/len(tf["prediction_percentile"].tolist())
        '''
    #print("warning: -1 is returned")
    return -1.0

def predict_binding_pid(x):#Transcript_Id,AAChange,patient_id):
    #print(x)
    #print("predict binding TID=%s, AAC=%s, PID=%s."%(Transcript_Id,AAChange,patient_id))
    Transcript_Id=x["Tid"]
    AAChange=x["AAC"]
    patient_id=x["Sample_Id"]
    for i in SixHLAs:
        x["Rank_"+i]=predict_binding(Transcript_Id, AAChange, x[i])
        x["Rank_"+i] = min(x["Rank_"+i] , 4.0)
    
    predicts = [ x["Rank_"+i] for i in SixHLAs]
    inverses = [1.0/p  for p in predicts if p>0]
    #print 
    #print (predicts)
    #print (inverses)
    if len(inverses)<1: return x
    x["Rank"]=len(inverses)/sum(inverses)
    return x

predict_binding("ENST00000400318","p.E104K","A*11:01")


-1.0

In [14]:
def read(cancer_type,rna_t=1000):
    Fil_SNV=0
    minus_GL_path=""
    sample_dir=""
    rna_path=""

    for p in glob.glob(cancer_type+"/*"):
        if ("minus_germline" in p):
            minus_GL_path=p
        elif ("Mutation" in p) and ("Calls" in p):
            sample_dir=p
        elif ("illuminahiseq" in p):
            rna_path=p
    if (minus_GL_path == "" or sample_dir == ""): raise Exception("File missing")

    rna_data=read_rna(rna_path)

    number_files=len(glob.glob(sample_dir+"/*"))
    tfs=[]
    for i,path in enumerate(glob.glob(sample_dir+"/*")):
        #path="BLCA/gdac.broadinstitute.org_BLCA.Mutation_Packager_Calls.Level_3.2016012800.0.0/TCGA-DK-A3IS-01.maf.txt"
        if ("MANIFEST" in path):continue
        print("%d/%d : "%(i,number_files),path)
        try:
            tf=read_and_filter(minus_GL_path,path,cancer_type)
        except Exception as e:
            print(e)
            continue
        #if (tf==None): break
        Fil_SNV+=(tf[tf["CCF"]>1.0].shape[0])
        tf=tf[tf["CCF"]<=1.0]
        if(tf.shape[0]==0): continue
        #print( tf[:5])
        #x=tf.iloc[0]
        #print (x)
        #a=obtain_rna_data(rna_data,x["Sample_Id"],hugo_symbol=x["Hugo_Symbol"],gene_id=x["Entrez_Gene_Id"])
        
        tf["RNA"] = tf.apply( (lambda x: obtain_rna_data(rna_data,x["Sample_Id"],hugo_symbol=x["Hugo_Symbol"],gene_id=x["Entrez_Gene_Id"]) ),axis=1 )
        
        for i in SixHLAs:
            tf["Rank_"+i]=-1.0
        tf["Rank"]=-1.0
        
        tf=tf.apply(predict_binding_pid,axis=1)
        #tf["binding"]=tf.apply( (lambda x:predict_binding_pid(str(x["Tid"]),x["AAC"],patient_id=x["Sample_Id"])), axis=1 )
        #tf=tf[tf["binding"]>=0]
        tfs.append(tf)
    return pd.concat(tfs)



In [15]:
# T="BRCA"
# tf=read(T)
# tf.to_csv("Results3/"+T+".csv")

In [ ]:
Ts=["ACC","BLCA","BRCA",
#Ts=[
"HNSC",
"KIRP",
"LIHC",
"LUAD",
"LUSC",
"PRAD",
"SKCM",
"STES"
]

ETs=[]
for T in Ts:
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            tf=read(T)
    except Exception as inst:
        ETs.append(T)
        print(inst)
        continue
    tf.to_csv("Results4/"+T+".csv")

0/91 :  ACC/gdac.broadinstitute.org_ACC.Mutation_Packager_Calls.Level_3.2016012800.0.0/TCGA-OR-A5JM-01.maf.txt
reading TCGA-OR-A5JM-01


Using TensorFlow backend.
W0821 16:11:01.598854 140371754637120 deprecation_wrapper.py:119] From /home/yq7/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0821 16:11:01.623940 140371754637120 deprecation_wrapper.py:119] From /home/yq7/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0821 16:11:01.779634 140371754637120 deprecation_wrapper.py:119] From /home/yq7/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0821 16:11:01.782033 140371754637120 deprecation_wrapper.py:119] From /home/yq7/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.Con

In [ ]:
tf[:5]